## Sistema de Avaliação de Risco de Crédito

1.0 Importe das bibliotecas

In [4]:
# Manipulação e visualização
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

# Opções de display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Filtro de alarmes
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("rawdata.csv", sep=";")
df
# Uma primeira olhada nos dados, visualizando todas as colunas.

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no


### Dados
O conjunto de dados contém informações sobre os clientes, incluindo dados demográficos, histórico financeiro e interações anteriores com o banco. A variável de destino, "y", indica se o cliente contratou um depósito a prazo (sim/não).

### Estrutura
4521 linhas × 17 colunas.

### Características
- idade: idade do cliente.
- emprego: ocupação ou cargo do cliente.
- estado civil: estado civil do cliente (casado, solteiro, divorciado).
- escolaridade: nível de escolaridade alcançado pelo cliente.
- inadimplência: indica se o cliente possui histórico de inadimplência em empréstimos (sim, não).
- saldo: saldo total da conta do cliente.
- moradia: indica se o cliente possui financiamento imobiliário (sim/não).
- empréstimo: indica se o cliente possui um empréstimo pessoal (sim/não).
- contato: método de comunicação com o cliente (celular, telefone).
- dia: último dia em que o cliente foi contatado.
- mês: último mês em que o cliente foi contatado.
- duração: duração do último contato com o cliente (em segundos).
- campanha: número de contatos feitos com o cliente nesta campanha.
- dias: número de dias desde o último contato com o cliente (-1 se o cliente não tiver sido contatado anteriormente).
- anterior: número de contatos feitos com este cliente antes desta campanha. poutcome: resultado da campanha de marketing anterior (sucesso, fracasso, desconhecido).
- y: variável de destino, indicando se o cliente contratou um depósito a prazo (sim/não).


Abaixo, seguem as informações de cada coluna do dataset:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   age        int64 
 1   job        object
 2   marital    object
 3   education  object
 4   default    object
 5   balance    int64 
 6   housing    object
 7   loan       object
 8   contact    object
 9   day        int64 
 10  month      object
 11  duration   int64 
 12  campaign   int64 
 13  pdays      int64 
 14  previous   int64 
 15  poutcome   object
 16  y          object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


Agora, o resumo estatístico das colunas numéricas do dataset:

In [4]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


Quantidade de nulos no dataset:

In [5]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

Não há nulos no dataset. Há duplicados?

In [6]:
df.duplicated().sum()

np.int64(0)